In [28]:
import pandas as pd
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score, KFold
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

# Read data from file 
file_path = r'D:\00. MSc Data Analytic\CIND820\Data\Crop data\Final_Data.xlsx'
weather_df = pd.read_excel(file_path, sheet_name='Weather')
crop_df = pd.read_excel(file_path, sheet_name='Crop')

cols = ['Year', 'County', 'Station Name', 'Climate ID', 'Date/Time', 'Month', 'Day', 'Max Temp (°C)', 'Min Temp (°C)',
        'Mean Temp (°C)', 'Heat Deg Days (°C)', 'Cool Deg Days (°C)', 'Total Rain (mm)', 'Total Snow (cm)',
        'Total Precip (mm)', 'Snow on Grnd (cm)', 'Dir of Max Gust (1s deg)', 'Spd of Max Gust (km/h)',
        'Max Temp Flag', 'Min Temp Flag', 'Mean Temp Flag', 'Heat Deg Days Flag', 'Cool Deg Days Flag',
        'Total Rain Flag', 'Total Snow Flag', 'Total Precip Flag', 'Snow on Grnd Flag', 'Dir of Max Gust Flag',
        'Spd of Max Gust Flag']
subset_df = weather_df[cols]

# Perform one-hot encoding on the categorical variables in Weather dataset:
for col in ['Max Temp Flag', 'Min Temp Flag', 'Mean Temp Flag', 'Heat Deg Days Flag', 'Cool Deg Days Flag']:
    onehot_df = pd.get_dummies(subset_df[col], prefix=col)
    subset_df = pd.concat([subset_df, onehot_df], axis=1)
    subset_df.drop(col, axis=1, inplace=True)

for col in ['Total Rain Flag', 'Total Snow Flag', 'Total Precip Flag', 'Snow on Grnd Flag', 'Dir of Max Gust Flag',
            'Spd of Max Gust Flag']:
    onehot_df = pd.get_dummies(subset_df[col], prefix=col)
    subset_df = pd.concat([subset_df, onehot_df], axis=1)
    subset_df.drop(col, axis=1, inplace=True)

subset_df = subset_df.drop(['Station Name', 'Climate ID','Date/Time'], axis=1)

In [29]:
# Drop the attribute columns with full data
subset_df.drop(['Year', 'County', 'Month', 'Day'], axis=1, inplace=True)

# Impute missing values using regression imputation
imputer = IterativeImputer(random_state=0, max_iter=100)
imputed_df = pd.DataFrame(imputer.fit_transform(subset_df), columns=subset_df.columns)

# Combine the attribute columns and imputed columns into a single dataframe
output_df = pd.concat([weather_df[['Year', 'County', 'Month', 'Day']], imputed_df], axis=1)

c:\Users\lupin\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\impute\_iterative.py:785: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


In [31]:
# Group the data by County and Year and aggregate the data
grouped_df = output_df.groupby(['County', 'Year']).agg({
    'Max Temp (°C)': ['mean', 'max', 'min', 'std'],
    'Min Temp (°C)': ['mean', 'max', 'min', 'std'],
    'Mean Temp (°C)': ['mean', 'max', 'min', 'std'],
    'Heat Deg Days (°C)': ['mean', 'max', 'min', 'std'],
    'Cool Deg Days (°C)': ['mean', 'max', 'min', 'std'],
    'Total Rain (mm)': ['mean', 'max', 'min', 'std'],
    'Total Snow (cm)': ['mean', 'max', 'min', 'std'],
    'Total Precip (mm)': ['mean', 'max', 'min', 'std'],
    'Snow on Grnd (cm)': ['mean', 'max', 'min', 'std'],
    'Dir of Max Gust (1s deg)': ['mean', 'max', 'min', 'std'],
    'Spd of Max Gust (km/h)': ['mean', 'max', 'min', 'std'],
    'Max Temp Flag_E': 'sum',
    'Max Temp Flag_M': 'sum',
    'Min Temp Flag_E': 'sum',
    'Min Temp Flag_M': 'sum',
    'Mean Temp Flag_E': 'sum',    
    'Mean Temp Flag_M': 'sum',
    'Heat Deg Days Flag_E': 'sum',
    'Heat Deg Days Flag_M': 'sum',
    'Cool Deg Days Flag_E': 'sum',
    'Cool Deg Days Flag_M': 'sum',
    'Total Rain Flag_A': 'sum',
    'Total Rain Flag_C': 'sum',
    'Total Rain Flag_E': 'sum',
    'Total Rain Flag_F': 'sum',
    'Total Rain Flag_L': 'sum',
    'Total Rain Flag_M': 'sum',
    'Total Rain Flag_T': 'sum',
    'Total Snow Flag_A': 'sum',
    'Total Snow Flag_C': 'sum',
    'Total Snow Flag_E': 'sum',
    'Total Snow Flag_F': 'sum',
    'Total Snow Flag_M': 'sum',
    'Total Snow Flag_T': 'sum',
    'Total Precip Flag_A': 'sum',
    'Total Precip Flag_C': 'sum',
    'Total Precip Flag_E': 'sum',
    'Total Precip Flag_F': 'sum',
    'Total Precip Flag_M': 'sum',
    'Total Precip Flag_T': 'sum',
    'Snow on Grnd Flag_E': 'sum',
    'Snow on Grnd Flag_M': 'sum',
    'Snow on Grnd Flag_T': 'sum',
    'Dir of Max Gust Flag_E': 'sum',
    'Dir of Max Gust Flag_M': 'sum',
    'Spd of Max Gust Flag_E': 'sum',
    'Spd of Max Gust Flag_M': 'sum'
})

# Flatten the resulting column names by joining the multi-index columns
grouped_df.columns = ['_'.join(col).strip() for col in grouped_df.columns.values]
grouped_df = grouped_df.reset_index()

In [33]:
# Merge the DataFrames on the "County" and "Year" columns, then clear all record with no Production
merged_df = pd.merge(grouped_df, crop_df, on=["County", "Year"])
merged_df["Yield (kg/Acres seeded)"] = merged_df.apply(lambda row: (row["Sum of Production ('000 tonnes)"] * 1000) / row["Sum of Acres seeded"] if row["Sum of Acres seeded"] != 0 else 0, axis=1)
merged_df = merged_df.drop(merged_df[merged_df["Yield (kg/Acres seeded)"] == 0].index)
merged_df = merged_df.drop(["Sum of Production ('000 tonnes)", "Sum of Acres seeded"], axis=1)

In [34]:
#remove County, Product and Year from data
merged_df = merged_df.drop(["County","Year","Product"], axis=1)

# Split the dataset into features (X) and target variable (y) and define which models use
X = merged_df.drop(['Yield (kg/Acres seeded)'], axis=1)
y = merged_df['Yield (kg/Acres seeded)']

dt_regressor = DecisionTreeRegressor(random_state=42)
svr_regressor = SVR(kernel='rbf')

# Define the cross-validation method
kfold = KFold(n_splits=10, shuffle=True, random_state=42)

# Train and evaluate the Decision Tree model
dt_scores = cross_val_score(dt_regressor, X, y, scoring='neg_mean_squared_error', cv=kfold)
dt_mse = -dt_scores.mean()

# Train and evaluate the Support Vector Machines model
svr_scores = cross_val_score(svr_regressor, X, y, scoring='neg_mean_squared_error', cv=kfold)
svr_mse = -svr_scores.mean()

#print out initial results
print('Decision Tree MSE:', dt_mse)
print('SVM MSE:', svr_mse)

Decision Tree MSE: 34214.95047265146
SVM MSE: 27029.863705540407
